In [1]:
# Use Random forest to classify MNIST images

from __future__ import print_function

import tensorflow as tf
from tensorflow.python.ops import resources
from tensorflow.contrib.tensor_forest.python import tensor_forest

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data", one_hot=False)

W0815 17:28:23.208465 140261121079168 deprecation.py:323] From <ipython-input-1-59973f381b33>:10: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0815 17:28:23.209789 140261121079168 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0815 17:28:23.211518 140261121079168 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow/contrib/learn/python/learn/datasets/base.py:252: wrapped_fn (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/data/train-images-idx3-ubyte.gz


W0815 17:28:23.596537 140261121079168 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0815 17:28:23.676176 140261121079168 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: __init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [0]:
# Hyper-parameters
num_steps = 500 # total number of steps to train
batch_size = 1024 # number of samples per batch
num_classes = 10 # number of output classes for 0-9 digits
num_features = 784 # each image is 28x28
num_trees = 10
max_nodes = 1000

In [0]:
# input and target data
X = tf.placeholder(tf.float32, shape=[None, num_features])
Y = tf.placeholder(tf.int32, shape=[None])

#Random Forest parameters
hparams = tensor_forest.ForestHParams(num_classes=num_classes,
                                     num_features=num_features,
                                     num_trees = num_trees,
                                     max_nodes = max_nodes).fill()

In [4]:
# build the Random Forest
forest_graph = tensor_forest.RandomForestGraphs(hparams)

# get training graph and loss
train_op = forest_graph.training_graph(X,Y)
loss_op = forest_graph.training_loss(X,Y)

# Make accuracy measurements
infer_op, _, _ = forest_graph.inference_graph(X)
correct_prediction = tf.equal(tf.argmax(infer_op,1), tf.cast(Y, tf.int64))
accuracy_op = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# initialize the variables
init = tf.group(tf.global_variables_initializer(),
                resources.initialize_resources(resources.shared_resources()))

W0815 17:28:23.931164 140261121079168 deprecation.py:506] From /usr/local/lib/python2.7/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling __init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0815 17:28:24.258132 140261121079168 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow/contrib/tensor_forest/python/tensor_forest.py:529: div (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


In [5]:
# start tensorflow session
with tf.train.MonitoredSession() as sess:
  sess.run(init)
  
  #Training
  for i in range(1, num_steps+1) :
    # prepare data
    
    # get the next batch of MNIST data (only images are needed not labels)
    batch_x, batch_y = mnist.train.next_batch(batch_size)

    _, l = sess.run([train_op, loss_op], feed_dict={X:batch_x, Y:batch_y})
    if i%50 ==0 or i==1:
      acc = sess.run(accuracy_op, feed_dict={X:batch_x, Y:batch_y})
      print("Step",i,"Loss", l, "Accuracy",acc)
      
  # Test the model
  test_x, test_y = mnist.test.images, mnist.test.labels
  print("Test accuracy",sess.run(accuracy_op, feed_dict={X:test_x, Y:test_y})) 

W0815 17:28:24.340109 140261121079168 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow/python/ops/array_ops.py:1354: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Step 1 Loss -1.0 Accuracy 0.4404297
Step 50 Loss -253.6 Accuracy 0.8701172
Step 100 Loss -540.0 Accuracy 0.9003906
Step 150 Loss -833.0 Accuracy 0.92089844
Step 200 Loss -1001.0 Accuracy 0.9199219
Step 250 Loss -1001.0 Accuracy 0.94140625
Step 300 Loss -1001.0 Accuracy 0.9345703
Step 350 Loss -1001.0 Accuracy 0.92871094
Step 400 Loss -1001.0 Accuracy 0.92871094
Step 450 Loss -1001.0 Accuracy 0.9277344
Step 500 Loss -1001.0 Accuracy 0.92285156
Test accuracy 0.921
